# Analyse Discriminante Factorielle Probabiliste

Ce notebook implémente une Analyse Discriminante Factorielle (ADF) probabiliste sur un jeu de données de notes d'étudiants répartis en trois sections : sciences, littérature et économie.

L'ADF probabiliste permet de :
1. Classifier les observations dans différentes classes
2. Estimer la probabilité d'appartenance à chaque classe
3. Projeter les données sur des axes discriminants pour visualisation
4. Interpréter l'importance relative des variables explicatives

Nous suivrons la méthode probabiliste avec :
- La formule de Bayes pour les probabilités a priori et a posteriori
- L'hypothèse de distribution normale multivariée pour chaque classe
- Les fonctions discriminantes linéaires pour la classification
- La projection sur les axes discriminants pour la visualisation

In [ ]:
# Import des bibliothèques nécessaires
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Configuration pour améliorer la visualisation
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

## Étape 1 : Préparation des données

Nous commençons par créer notre jeu de données à partir du tableau fourni et effectuer un prétraitement initial.

In [ ]:
# Création du DataFrame à partir des données de l'image
donnees = {
    'section': ['sciences', 'sciences', 'sciences', 'sciences', 'sciences',
                'litterature', 'litterature', 'litterature', 'litterature', 'litterature',
                'economie', 'economie', 'economie', 'economie', 'economie'],
    'mathematiques': [15, 14, 16, 13, 15, 10, 11, 9, 10, 8, 12, 13, 14, 12, 11],
    'economie_generale': [13, 12, 14, 11, 13, 15, 16, 17, 15, 16, 16, 15, 17, 16, 15],
    'francais': [12, 13, 14, 12, 15, 16, 15, 17, 14, 18, 13, 12, 11, 14, 13]
}

# Création du DataFrame
df = pd.DataFrame(donnees)

# Affichage du DataFrame et statistiques descriptives
print("Aperçu des données :")
display(df.head())

print("\nStatistiques descriptives par section :")
display(df.groupby('section').agg(['mean', 'std']))

# Conversion des classes en numérique pour faciliter les calculs
# Chaque section (classes) sera représentée par un index numérique
sectionUnique = df['section'].unique()
mapSection = {section: i for i, section in enumerate(sectionUnique)}
df['sectionNum'] = df['section'].map(mapSection)

print("\nCorrespondance des sections avec leurs indices:")
for section, num in mapSection.items():
    print(f"{section}: {num}")

## Étape 2 : Extraction et normalisation des variables

Nous extrayons les variables explicatives et la variable cible, puis normalisons les données pour les rendre comparables.

In [ ]:
# Extraction des variables explicatives (X) et de la variable cible (y)
X = df[['mathematiques', 'economie_generale', 'francais']].values
y = df['sectionNum'].values

# Normalisation des données (centrage-réduction)
# Pour chaque variable, on soustrait la moyenne et divise par l'écart-type
# La normalisation est importante en ADF pour donner un poids comparable à chaque variable
Xmoyenne = np.mean(X, axis=0)  # Moyenne de chaque variable
XecartType = np.std(X, axis=0)  # Écart-type de chaque variable
Xnormalise = (X - Xmoyenne) / XecartType  # Normalisation (z-score)

print("Variables avant normalisation (premières lignes):")
print(X[:5])
print("\nVariables après normalisation (premières lignes):")
print(Xnormalise[:5])
print("\nMoyennes des variables normalisées (doit être proche de zéro):")
print(np.mean(Xnormalise, axis=0))
print("\nÉcarts-types des variables normalisées (doit être proche de un):")
print(np.std(Xnormalise, axis=0))

## Étape 3 : Calcul des probabilités a priori (Formule de Bayes)

La formule de Bayes nous permet de calculer la probabilité P(C_k | x) à partir de P(x | C_k) et P(C_k).
Nous commençons par calculer les probabilités a priori P(C_k) pour chaque classe.

In [ ]:
# Récupération des classes uniques et initialisation des compteurs
classes = np.unique(y)        # Identifiants numériques des classes [0, 1, 2]
nbClasses = len(classes)      # Nombre de classes (3 dans ce cas)
nbEchantillons = len(y)       # Nombre total d'observations

# Calcul des probabilités a priori P(C_k) pour chaque classe
# La probabilité a priori est la proportion d'observations dans chaque classe
probsPrior = np.zeros(nbClasses)  # Initialisation du vecteur des probabilités
for i, cls in enumerate(classes):
    # Nombre d'observations dans la classe / Nombre total d'observations
    probsPrior[i] = np.sum(y == cls) / nbEchantillons

print("Probabilités a priori P(C_k):")
for i, cls in enumerate(sectionUnique):
    print(f"{cls}: {probsPrior[i]:.3f}")

# Vérification que les probabilités a priori somment à 1
print(f"\nSomme des probabilités a priori: {np.sum(probsPrior):.3f} (doit être égale à 1)")

## Étape 4 : Calcul des moyennes par classe et de la matrice de covariance commune

Pour modéliser P(x | C_k) avec une distribution normale multivariée, nous avons besoin :
1. Des vecteurs moyens μ_k pour chaque classe
2. D'une matrice de covariance commune Σ partagée par toutes les classes (hypothèse d'homoscédasticité)

In [ ]:
# Calcul des moyennes par classe (μ_k)
moyennes = np.zeros((nbClasses, X.shape[1]))  # Matrice pour stocker les moyennes (nbClasses × nbVariables)
for i, cls in enumerate(classes):
    # Moyenne des observations normalisées de la classe k
    moyennes[i] = np.mean(Xnormalise[y == cls], axis=0)

print("Moyennes par classe (après normalisation):")
for i, cls in enumerate(sectionUnique):
    print(f"{cls}: {moyennes[i]}")

# Calcul de la matrice de dispersion intra-classe (W)
# Cette matrice représente la variabilité à l'intérieur de chaque classe
matriceDispersionIntra = np.zeros((X.shape[1], X.shape[1]))  # Initialisation (nbVariables × nbVariables)
for i, cls in enumerate(classes):
    donneesClasse = Xnormalise[y == cls]           # Sélection des observations de la classe
    donneesCentrees = donneesClasse - moyennes[i]  # Centrage par rapport à la moyenne de la classe
    # Calcul de la matrice de dispersion pour la classe et ajout au total
    matriceDispersionIntra += np.dot(donneesCentrees.T, donneesCentrees)

# Calcul de la matrice de covariance commune (pooled covariance)
# On divise par (n - K) où n est le nombre total d'observations et K le nombre de classes
covarianceCommune = matriceDispersionIntra / (nbEchantillons - nbClasses)

print("\nMatrice de covariance commune (pooled):")
print(covarianceCommune)

# Vérification que la matrice de covariance est inversible
try:
    # On tente de calculer l'inverse de la matrice de covariance
    # Cette étape est cruciale car les fonctions discriminantes en dépendent
    covarianceCommuneInv = np.linalg.inv(covarianceCommune)
except np.linalg.LinAlgError:
    # Si la matrice n'est pas inversible, on ajoute un petit terme de régularisation
    print("\nAttention: La matrice de covariance n'est pas inversible. Ajout d'un petit epsilon.")
    covarianceCommune += np.eye(covarianceCommune.shape[0]) * 1e-6
    covarianceCommuneInv = np.linalg.inv(covarianceCommune)
    print("Matrice régularisée et inversée avec succès.")
else:
    print("\nMatrice de covariance inversée avec succès.")

## Étape 5 : Calcul des fonctions discriminantes linéaires

La fonction discriminante δ_k(x) pour chaque classe est définie comme :

δ_k(x) = x^T Σ^(-1) μ_k - 0.5 μ_k^T Σ^(-1) μ_k + log(P(C_k))

Cette formule est une transformation mathématique de la densité de probabilité qui simplifie la classification.

In [ ]:
# Calcul des fonctions discriminantes linéaires pour chaque observation et chaque classe
# δ_k(x) = x^T Σ^(-1) μ_k - 0.5 μ_k^T Σ^(-1) μ_k + log(P(C_k))
scoresDiscriminants = np.zeros((nbEchantillons, nbClasses))  # Matrice pour stocker les scores

for i in range(nbEchantillons):  # Pour chaque observation
    for k, cls in enumerate(classes):  # Pour chaque classe
        # Terme 1: x^T Σ^(-1) μ_k (produit scalaire pondéré par l'inverse de la covariance)
        terme1 = np.dot(np.dot(Xnormalise[i], covarianceCommuneInv), moyennes[k])
        
        # Terme 2: 0.5 μ_k^T Σ^(-1) μ_k (distance de Mahalanobis au carré du centre de la classe à l'origine)
        terme2 = 0.5 * np.dot(np.dot(moyennes[k], covarianceCommuneInv), moyennes[k])
        
        # Terme 3: log(P(C_k)) (logarithme de la probabilité a priori)
        terme3 = np.log(probsPrior[k]) if probsPrior[k] > 0 else -np.inf
        
        # Score discriminant = terme1 - terme2 + terme3
        scoresDiscriminants[i, k] = terme1 - terme2 + terme3

# Affichage des scores discriminants pour quelques observations
print("Scores discriminants pour les 5 premières observations:")
for i in range(min(5, nbEchantillons)):
    print(f"Observation {i+1}:")
    for k, cls in enumerate(sectionUnique):
        print(f"  {cls}: {scoresDiscriminants[i, k]:.4f}")

## Étape 6 : Calcul des probabilités a posteriori et classification

Les probabilités a posteriori P(C_k | x) sont proportionnelles à exp(δ_k(x)). 
Nous normalisons ces valeurs pour obtenir des probabilités valides qui somment à 1.

La règle de classification attribue chaque observation à la classe avec le score discriminant le plus élevé.

In [ ]:
# Conversion des scores discriminants en probabilités a posteriori
# P(C_k|x) ∝ exp(δ_k(x))
probsPosterior = np.exp(scoresDiscriminants)  # Exponentielle des scores

# Normalisation pour que les probabilités somment à 1 pour chaque observation
probsPosterior = probsPosterior / np.sum(probsPosterior, axis=1, keepdims=True)

# Classification: attribution à la classe avec le score discriminant le plus élevé
# Règle de décision: Classe(x) = arg max_k δ_k(x)
classesPredites = np.argmax(scoresDiscriminants, axis=1)  # Indice de la classe avec le score max
sectionsPredites = [sectionUnique[i] for i in classesPredites]  # Conversion en nom de section

# Création d'un DataFrame avec les résultats
dfResultats = pd.DataFrame({
    'sectionReelle': df['section'],  # Section réelle
    'sectionPredite': sectionsPredites  # Section prédite par le modèle
})

# Ajout des probabilités a posteriori pour chaque classe
for k, cls in enumerate(sectionUnique):
    dfResultats[f'proba_{cls}'] = probsPosterior[:, k]

print("Résultats de classification avec probabilités a posteriori:")
display(dfResultats)

# Évaluation: matrice de confusion
# La matrice de confusion montre combien d'observations de chaque classe réelle 
# ont été classées dans chaque classe prédite
matriceConfusion = pd.crosstab(
    df['section'],  # Classes réelles (lignes)
    pd.Series(sectionsPredites, name='Prédit'),  # Classes prédites (colonnes)
    margins=True,  # Ajouter les sommes marginales
    normalize='index'  # Normaliser par ligne (% par classe réelle)
)

print("\nMatrice de confusion (pourcentages par classe réelle):")
display(matriceConfusion)

# Calcul du taux de bon classement global
precision = np.sum(df['section'] == pd.Series(sectionsPredites)) / len(df)
print(f"\nTaux de bon classement global: {precision:.2%}")

## Étape 7 : Analyse des axes discriminants

Nous calculons maintenant les axes discriminants qui maximisent la séparation entre les classes.
Ces axes sont les vecteurs propres du produit Σ^(-1) * S_B, où S_B est la matrice de dispersion inter-classes.

In [ ]:
# Calcul de la matrice de dispersion inter-classes (B)
# Cette matrice représente la variabilité entre les centres des classes
matriceDispersionInter = np.zeros((X.shape[1], X.shape[1]))  # Initialisation
moyenneGlobale = np.mean(Xnormalise, axis=0)  # Moyenne globale des données normalisées

for i, cls in enumerate(classes):
    nbEchantillonsClasse = np.sum(y == cls)  # Nombre d'observations dans la classe
    diffMoyenne = moyennes[i] - moyenneGlobale  # Écart entre moyenne de classe et moyenne globale
    # Matrice d'écart pondérée par le nombre d'observations
    matriceDispersionInter += nbEchantillonsClasse * np.outer(diffMoyenne, diffMoyenne)

# Résolution du problème des valeurs propres généralisées: Σ^(-1) * S_B
# Les valeurs propres représentent l'importance de chaque axe discriminant
# Les vecteurs propres sont les directions des axes discriminants dans l'espace original
valeursPropres, vecteursPropres = np.linalg.eig(np.dot(covarianceCommuneInv, matriceDispersionInter))

# Tri des valeurs propres et vecteurs propres par ordre décroissant
idx = valeursPropres.argsort()[::-1]  # Indices des valeurs propres triées
valeursPropres = valeursPropres[idx]  # Réorganisation des valeurs propres
vecteursPropres = vecteursPropres[:, idx]  # Réorganisation des vecteurs propres

# Nombre de composantes discriminantes à conserver (au plus nbClasses - 1)
nbComposantes = min(nbClasses - 1, X.shape[1])
vecteursPropres = vecteursPropres[:, :nbComposantes]  # Ne garder que les nbComposantes premiers vecteurs

# Calcul du pourcentage de variance expliquée par chaque axe discriminant
ratioVarianceExpliquee = valeursPropres[:nbComposantes] / np.sum(valeursPropres)

print("Valeurs propres (importance des axes discriminants):")
for i in range(nbComposantes):
    print(f"Axe {i+1}: {valeursPropres[i]:.4f} ({ratioVarianceExpliquee[i]:.2%} de variance expliquée)")

# Coefficients discriminants (vecteurs propres standardisés)
# Ces coefficients montrent la contribution de chaque variable originale aux axes discriminants
dfCoefficients = pd.DataFrame(
    vecteursPropres,
    index=['mathematiques', 'economie_generale', 'francais'],
    columns=[f'Axe {i+1}' for i in range(nbComposantes)]
)

print("\nCoefficients discriminants:")
display(dfCoefficients)

# Projection des données sur les axes discriminants
# Cette projection permet de visualiser les données dans l'espace discriminant
Xprojete = np.dot(Xnormalise, vecteursPropres)

## Étape 8 : Visualisation des données projetées

Nous projetons les données sur les axes discriminants et visualisons la séparation entre les classes.

In [ ]:
# Visualisation des données projetées sur les axes discriminants
plt.figure(figsize=(12, 8))
couleurs = ['blue', 'red', 'green']  # Couleurs pour les différentes classes
marqueurs = ['o', 's', '^']  # Marqueurs pour les différentes classes

# Scatter plot des projections avec coloration par classe
for i, cls in enumerate(sectionUnique):
    idx = df['section'] == cls  # Indices des observations de la classe
    plt.scatter(
        Xprojete[idx, 0],  # Coordonnées sur l'axe 1
        Xprojete[idx, 1] if nbComposantes > 1 else np.zeros(np.sum(idx)),  # Coordonnées sur l'axe 2
        c=couleurs[i],  # Couleur de la classe
        marker=marqueurs[i],  # Marqueur de la classe
        label=cls,  # Légende
        alpha=0.7,  # Transparence
        s=80  # Taille des points
    )

# Configuration du graphique
plt.title('Projection des données sur les axes discriminants', fontsize=14)
plt.xlabel(f'Axe discriminant 1 ({ratioVarianceExpliquee[0]:.2%} variance expliquée)', fontsize=12)
if nbComposantes > 1:
    plt.ylabel(f'Axe discriminant 2 ({ratioVarianceExpliquee[1]:.2%} variance expliquée)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)

# Affichage des centroïdes des classes
for i, cls in enumerate(sectionUnique):
    idx = df['section'] == cls
    centroideX = np.mean(Xprojete[idx, 0])  # Coordonnée x du centroïde
    centroideY = np.mean(Xprojete[idx, 1]) if nbComposantes > 1 else 0  # Coordonnée y du centroïde
    
    # Représentation du centroïde
    plt.scatter(
        centroideX, 
        centroideY, 
        s=200,  # Taille plus grande pour le centroïde
        c=couleurs[i], 
        marker='*',  # Étoile pour les centroïdes
        edgecolor='black',  # Contour noir
        linewidth=1.5  # Épaisseur du contour
    )
    
    # Étiquette du centroïde
    plt.annotate(
        cls,  # Texte
        (centroideX, centroideY),  # Position
        fontsize=14,  # Taille de police
        ha='center',  # Alignement horizontal
        va='center',  # Alignement vertical
        bbox=dict(facecolor='white', alpha=0.5, pad=5)  # Fond semi-transparent
    )

plt.tight_layout()
plt.show()

## Étape 9 : Visualisation des frontières de décision

Nous visualisons les frontières de décision entre les classes dans l'espace discriminant.

In [ ]:
# Visualisation des frontières de décision (si on a au moins 2 axes discriminants)
if nbComposantes >= 2:
    plt.figure(figsize=(12, 10))
    
    # Définition d'une grille pour visualiser les frontières
    xMin, xMax = Xprojete[:, 0].min() - 1, Xprojete[:, 0].max() + 1
    yMin, yMax = Xprojete[:, 1].min() - 1, Xprojete[:, 1].max() + 1
    xx, yy = np.meshgrid(
        np.arange(xMin, xMax, 0.1),  # Points sur l'axe x
        np.arange(yMin, yMax, 0.1)   # Points sur l'axe y
    )
    
    # Création des points de la grille
    pointsGrille = np.c_[xx.ravel(), yy.ravel()]  # Combinaison des coordonnées x et y
    scoresGrille = np.zeros((pointsGrille.shape[0], nbClasses))  # Scores pour chaque point et chaque classe
    
    # Calcul simplifié des scores discriminants pour les points de la grille
    # Note: C'est une approximation pour visualisation seulement
    for i in range(pointsGrille.shape[0]):
        for k, cls in enumerate(classes):
            # Calcul basé sur la distance aux centres projetés
            centreProjete = np.mean(Xprojete[y == k, :2], axis=0)
            distance = np.sum((pointsGrille[i] - centreProjete)**2)  # Distance euclidienne au carré
            scoresGrille[i, k] = -distance + np.log(probsPrior[k])  # Score approximatif
    
    # Classification des points de la grille
    predictionGrille = np.argmax(scoresGrille, axis=1)  # Classe avec le score maximum
    
    # Affichage des régions de décision
    plt.contourf(
        xx, yy,  # Coordonnées de la grille
        predictionGrille.reshape(xx.shape),  # Prédictions reformatées selon la grille
        alpha=0.4,  # Transparence
        cmap=plt.cm.brg  # Palette de couleurs
    )
    
    # Affichage des points
    for i, cls in enumerate(sectionUnique):
        idx = df['section'] == cls
        plt.scatter(
            Xprojete[idx, 0],  # Coordonnées sur l'axe 1
            Xprojete[idx, 1],  # Coordonnées sur l'axe 2
            c=couleurs[i],     # Couleur
            marker=marqueurs[i],  # Marqueur
            label=cls,  # Légende
            edgecolor='k',  # Contour noir
            s=80  # Taille
        )
    
    # Configuration du graphique
    plt.title('Frontières de décision de l\'Analyse Discriminante', fontsize=14)
    plt.xlabel(f'Axe discriminant 1 ({ratioVarianceExpliquee[0]:.2%})', fontsize=12)
    plt.ylabel(f'Axe discriminant 2 ({ratioVarianceExpliquee[1]:.2%})', fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## Étape 10 : Cercle des corrélations

Nous visualisons les corrélations entre les variables originales et les axes discriminants pour interpréter leur signification.

In [ ]:
# Cercle des corrélations entre variables originales et axes discriminants
plt.figure(figsize=(10, 10))

# Calcul des corrélations entre variables originales et axes discriminants
correlations = np.zeros((X.shape[1], nbComposantes))  # Matrice pour stocker les corrélations
for i in range(X.shape[1]):  # Pour chaque variable originale
    for j in range(nbComposantes):  # Pour chaque axe discriminant
        # Calcul de la corrélation linéaire
        correlations[i, j] = np.corrcoef(Xnormalise[:, i], Xprojete[:, j])[0, 1]

# Dessiner le cercle de corrélation
circle = plt.Circle((0, 0), 1, fill=False, color='gray', linestyle='--')
plt.gca().add_patch(circle)
plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)  # Axe horizontal
plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)  # Axe vertical

# Dessiner les vecteurs de variables
variables = ['mathematiques', 'economie_generale', 'francais']
for i, var_name in enumerate(variables):
    # Dessin de la flèche représentant la variable
    plt.arrow(
        0, 0,  # Point de départ (origine)
        correlations[i, 0], correlations[i, 1],  # Direction et longueur
        head_width=0.05,  # Largeur de la pointe de flèche
        head_length=0.05,  # Longueur de la pointe de flèche
        fc='blue',  # Couleur de remplissage
        ec='blue'   # Couleur du contour
    )
    
    # Étiquette de la variable
    plt.text(
        correlations[i, 0] * 1.15,  # Décalage en x pour lisibilité
        correlations[i, 1] * 1.15,  # Décalage en y pour lisibilité
        var_name,  # Nom de la variable
        color='blue',  # Couleur du texte
        ha='center',  # Alignement horizontal
        va='center',  # Alignement vertical
        fontsize=12   # Taille du texte
    )

# Configuration du graphique
plt.xlim(-1.1, 1.1)  # Limites de l'axe x
plt.ylim(-1.1, 1.1)  # Limites de l'axe y
plt.grid(True, alpha=0.3)  # Grille
plt.title('Cercle des corrélations', fontsize=14)  # Titre
plt.xlabel(f'Axe discriminant 1 ({ratioVarianceExpliquee[0]:.2%})', fontsize=12)  # Étiquette de l'axe x
plt.ylabel(f'Axe discriminant 2 ({ratioVarianceExpliquee[1]:.2%})', fontsize=12)  # Étiquette de l'axe y

plt.tight_layout()
plt.show()

## Conclusion et interprétation

L'analyse discriminante factorielle probabiliste nous a permis de :

1. Classifier correctement les étudiants dans leurs sections respectives
2. Calculer les probabilités d'appartenance à chaque section
3. Visualiser la séparation entre les sections sur les axes discriminants
4. Comprendre l'importance relative des matières dans la discrimination

### Interprétation des résultats :

- Le premier axe discriminant semble opposer les compétences en mathématiques aux compétences littéraires
- Le second axe discriminant est davantage lié aux compétences en économie et en français
- Les sections sont bien séparées dans l'espace discriminant, ce qui indique que le modèle est efficace
- Les centroïdes montrent clairement la position moyenne de chaque section par rapport aux axes discriminants

Cette analyse nous permet de comprendre comment les différentes matières contribuent à la discrimination entre les sections d'études, et pourrait être utilisée pour orienter de nouveaux étudiants vers la section la plus adaptée à leur profil.